Players dataset creation

The Fantacalcio players list is manually downloaded from https://www.fantacalcio.it/quotazioni-fantacalcio

Here, the players database is generated, by merging the Fantacalcio list to stats downloaded from http://fbref.com

In [31]:
import pandas as pd

Load fbref data for outfield players and goalkeepers.

Generate fbref player list, adding player surname (with special characters replaced to normal ones)

In [32]:
rcsv = pd.read_csv('fbref_data/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [33]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

keepers_ID = len(outfield_players)

players

,player,team
0,Michel Aebischer,Bologna
1,Luis Alberto,Lazio
2,Pontus Almqvist,Lecce
3,Lorenzo Amatucci,Fiorentina
4,Bruno Amione,Hellas Verona
...,...,...
419,Łukasz Skorupski,Bologna
420,Yann Sommer,Inter
421,Wojciech Szczęsny,Juventus
422,Pietro Terracciano,Fiorentina


In [34]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [35]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [36]:
print(players[['player', 'surname']].to_string())

                         player           surname
0              Michel Aebischer         Aebischer
1                  Luis Alberto           Alberto
2               Pontus Almqvist          Almqvist
3              Lorenzo Amatucci          Amatucci
4                  Bruno Amione            Amione
5               Felipe Anderson          Anderson
6                 Houssem Aouar             Aouar
7              Marko Arnautović        Arnautovic
8              Kristjan Asllani           Asllani
9               Tommaso Augello           Augello
10           Yann Aurel Bisseck           Bisseck
11                   Paulo Azzi              Azzi
12           Oussama El Azzouzi           Azzouzi
13                 Milan Badelj            Badelj
14                   Jaime Báez              Baez
15                Nedim Bajrami           Bajrami
16               Mitchel Bakker            Bakker
17             Tommaso Baldanzi          Baldanzi
18                 Lameck Banda             Banda


Replace the surname for some specific players, according to config/name_fix.txt file.

This is done for players for which the decoded fbref surname doesn't correspond to Fantacalcio list.

In [37]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix



Ostigard
Augusto
Kjaer
Djuric
Gudmundsson


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Inter
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


Load players from Fantacalcio list.

In [38]:
fc_data = pd.read_excel('fantacalcio/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players.loc[i, 'surname'] = spl[-2]
        fc_players.loc[i, 'initial'] = spl[-1][0]
    else:
        fc_players.loc[i, 'surname'] = spl[-1]
        fc_players.loc[i, 'initial'] = ''
    
fc_players



,id,r,name,team,surname,initial
0,4312,P,Maignan,Milan,Maignan,
1,572,P,Meret,Napoli,Meret,
2,2428,P,Sommer,Inter,Sommer,
3,453,P,Szczesny,Juventus,Szczesny,
4,2814,P,Provedel,Lazio,Provedel,
...,...,...,...,...,...,...
530,6242,A,Bidaoui,Frosinone,Bidaoui,
531,6418,A,Burnete,Lecce,Burnete,
532,6419,A,Corfitzen,Lecce,Corfitzen,
533,6427,A,Stewart,Salernitana,Stewart,


Associate players from Fantacalcio list to ID for FBref data.

In [39]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players.loc[i, 'fb_ID']  = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players.loc[i, 'fb_ID'] = j
                
        

Print players for which the association failed.

Most of them are players who didn't play a single Serie A game this season with their team. If that is the case, and there is data from their previous team, that is taken here.

Others are ones for which the FBRef surname doesn't correspond to Fantacalcio one.


For example, Cabral is Arthur for FBref.

Correction is made in the name_fix code above.

In [40]:
exceptions = ['pellegrini', 'bastoni'] # exceptions for such players that have the same surname as others (Berardi A., Luca Pellegrini)

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        found = False
        for j in range(players.shape[0]):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                if(not(players['surname'][j].lower() in exceptions)):
                    if((fc_players['r'][i] == 'P') == (j >= keepers_ID)):
                        fc_players.loc[i, 'fb_ID']= j
                        found = True
        if(found):
            print(fc_players['name'][i] + ' from previous team stats')
        else:
            print(fc_players['name'][i] + ' not found')

Carnesecchi not found
Cragno not found
Sportiello not found
Mirante not found
Sepe not found
Leali not found
Lamanna not found
Sommariva not found
Pegolo not found
Perilli not found
Padelli not found
Scuffet not found
Gollini not found
Audero not found
Di Gennaro not found
Pinsoglio not found
Aresti not found
Fiorillo not found
Rossi F. not found
Costil not found
Ravaglia F. not found
Frattali not found
Contini not found
Brancolini not found
Berardi A. not found
Gemello not found
Boer not found
Bagnolini not found
Svilar not found
Sorrentino A. not found
Martinelli T. not found
Popa not found
Stubljar not found
Gori not found
Borbei not found
Okoye not found
Mandas not found
Pavard not found
Acerbi not found
Toloi not found
N'dicka not found
Bastoni S. not found
Kristensen not found
Natan not found
Mina not found
Hateboer not found
Masina not found
Djidji not found
Tressoldi not found
Hatzidiakos not found
Calafiori not found
Ehizibue not found
Vogliacco not found
Ferrari G. not found


In [45]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,4312,P,Maignan,Milan,Maignan,,406,28-068,1995,0,...,30.5,20,65.0,49.3,43,12,27.9,3,1.00,10.3
1,572,P,Meret,Napoli,Meret,,408,26-171,1997,0,...,24.4,8,0.0,21.4,15,1,6.7,5,1.67,18.1
2,2428,P,Sommer,Inter,Sommer,,420,34-266,1988,0,...,31.4,21,23.8,31.0,35,1,2.9,0,0.00,6.8
3,453,P,Szczesny,Juventus,Szczesny,,421,33-144,1990,0,...,31.6,3,66.7,53.3,16,1,6.3,0,0.00,10.8
4,2814,P,Provedel,Lazio,Provedel,,416,29-176,1994,0,...,26.1,11,27.3,31.6,39,2,5.1,6,2.00,16.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
531,6418,A,Burnete,Lecce,Burnete,,51,19-221,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
532,6419,A,Corfitzen,Lecce,Corfitzen,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
533,6427,A,Stewart,Salernitana,Stewart,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Populate players dataset with stats from FBref, for outfield players and goalkeepers

In [46]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            fc_players.loc[i, columns_to_copy[j]] = outfield_players.loc[fc_players['fb_ID'][i], columns_to_copy[j]]
            

In [47]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [48]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]
            fc_players.loc[i, columns_to_copy[j]] = keeper_players.loc[fc_players['fb_ID'][i] - delta_k, columns_to_copy[j]]

In [49]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,4312,P,Maignan,Milan,Maignan,,406,28-068,1995,0,...,30.5,20,65.0,49.3,43,12,27.9,3,1.00,10.3
1,572,P,Meret,Napoli,Meret,,408,26-171,1997,0,...,24.4,8,0.0,21.4,15,1,6.7,5,1.67,18.1
2,2428,P,Sommer,Inter,Sommer,,420,34-266,1988,0,...,31.4,21,23.8,31.0,35,1,2.9,0,0.00,6.8
3,453,P,Szczesny,Juventus,Szczesny,,421,33-144,1990,0,...,31.6,3,66.7,53.3,16,1,6.3,0,0.00,10.8
4,2814,P,Provedel,Lazio,Provedel,,416,29-176,1994,0,...,26.1,11,27.3,31.6,39,2,5.1,6,2.00,16.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
531,6418,A,Burnete,Lecce,Burnete,,51,19-221,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
532,6419,A,Corfitzen,Lecce,Corfitzen,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
533,6427,A,Stewart,Salernitana,Stewart,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Load votes database, to add data to players database (mean vote and its standard deviation)

In [50]:
import numpy as np

votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)

Compute the average Serie A Goal Keeper mean vote and vote std

In [51]:
min_votes = 2 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

perf_df_P = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]): 
    if(fc_players.loc[i]['r'] == 'P'):
        v = np.array([])
        for j in range(votes.shape[0]):
            if(fc_players['name'][i] == votes['player'][j]):
                v = np.append(v, votes['vote'][j])

        if(v.shape[0] >= min_votes - 1):
            row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df_P.columns)
            perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


print(perf_df_P.mean())

perf_df_P


vote_avg    6.020000
vote_std    0.206692
dtype: float64


,vote_avg,vote_std
0,6.000000,0.000000
1,5.833333,0.235702
2,6.000000,0.000000
3,7.000000,0.000000
4,6.166667,0.235702
5,5.666667,0.471405
6,6.500000,0.707107
7,6.166667,0.235702
8,6.000000,0.408248
9,6.000000,0.000000


Add to players data their mean vote (and its standard deviation).

For players who don't have a minimum amount of games, more data to reach this value is computed, according to the average Serie A player vote (and std). For goalkeepers, this values are different.


In [52]:
min_votes = 1 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

#outfield players
mean_def = 6
std_def = 0.58

#goalkeepers
mean_def_P = 6.22
std_def_P = 0.43


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    mean_def_i = mean_def
    std_def_i = std_def
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

,vote_avg,vote_std
0,6.000000,0.000000
1,5.833333,0.235702
2,6.000000,0.000000
3,7.000000,0.000000
4,6.166667,0.235702
...,...,...
530,6.217118,0.000000
531,6.000000,0.000000
532,5.378980,0.000000
533,5.558549,0.000000


In [53]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,4312,P,Maignan,Milan,Maignan,,406,28-068,1995,0,...,65.0,49.3,43,12,27.9,3,1.00,10.3,6.000000,0.000000
1,572,P,Meret,Napoli,Meret,,408,26-171,1997,0,...,0.0,21.4,15,1,6.7,5,1.67,18.1,5.833333,0.235702
2,2428,P,Sommer,Inter,Sommer,,420,34-266,1988,0,...,23.8,31.0,35,1,2.9,0,0.00,6.8,6.000000,0.000000
3,453,P,Szczesny,Juventus,Szczesny,,421,33-144,1990,0,...,66.7,53.3,16,1,6.3,0,0.00,10.8,7.000000,0.000000
4,2814,P,Provedel,Lazio,Provedel,,416,29-176,1994,0,...,27.3,31.6,39,2,5.1,6,2.00,16.3,6.166667,0.235702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.217118,0.000000
531,6418,A,Burnete,Lecce,Burnete,,51,19-221,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.000000,0.000000
532,6419,A,Corfitzen,Lecce,Corfitzen,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.378980,0.000000
533,6427,A,Stewart,Salernitana,Stewart,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.558549,0.000000


In [54]:
GK_GAMES_COLUMN = 118

fc_players.columns[GK_GAMES_COLUMN]

# check it if is 'gk_games'

'gk_games'

For goalkeepers who didn't play a miminum amount of games, data is weightly averaged with one of the main goalkeeper of their same team.

In [55]:
min_gk_games = 1 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[GK_GAMES_COLUMN:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Carnesecchi, 0.0
Cragno, 0.0
Sportiello, 0.0
Mirante, 0.0
Sepe, 0.0
Leali, 0.0
Lamanna, 0.0
Sommariva, 0.0
Pegolo, 0.0
Perilli, 0.0
Padelli, 0.0
Scuffet, 0.0
Gollini, 0.0
Audero, 0.0
Di Gennaro, 0.0
Pinsoglio, 0.0
Aresti, 0.0
Fiorillo, 0.0
Rossi F., 0.0
Costil, 0.0
Ravaglia F., 0.0
Frattali, 0.0
Contini, 0.0
Brancolini, 0.0
Berardi A., 0.0
Gemello, 0.0
Boer, 0.0
Bagnolini, 0.0
Svilar, 0.0
Sorrentino A., 0.0
Martinelli T., 0.0
Popa, 0.0
Stubljar, 0.0
Gori, 0.0
Borbei, 0.0
Okoye, 0.0
Mandas, 0.0


In [56]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,4312,P,Maignan,Milan,Maignan,,406,28-068,1995,0,...,65.0,49.3,43,12,27.9,3,1.00,10.3,6.000000,0.000000
1,572,P,Meret,Napoli,Meret,,408,26-171,1997,0,...,0.0,21.4,15,1,6.7,5,1.67,18.1,5.833333,0.235702
2,2428,P,Sommer,Inter,Sommer,,420,34-266,1988,0,...,23.8,31.0,35,1,2.9,0,0.00,6.8,6.000000,0.000000
3,453,P,Szczesny,Juventus,Szczesny,,421,33-144,1990,0,...,66.7,53.3,16,1,6.3,0,0.00,10.8,7.000000,0.000000
4,2814,P,Provedel,Lazio,Provedel,,416,29-176,1994,0,...,27.3,31.6,39,2,5.1,6,2.00,16.3,6.166667,0.235702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.217118,0.000000
531,6418,A,Burnete,Lecce,Burnete,,51,19-221,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.000000,0.000000
532,6419,A,Corfitzen,Lecce,Corfitzen,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.378980,0.000000
533,6427,A,Stewart,Salernitana,Stewart,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.558549,0.000000


Save to file.

In [57]:
fc_players.to_excel('mid_outputs/players_stats.xlsx')